<a href="https://colab.research.google.com/github/pnbc/how-to-dp-fy-ml/blob/main/Label_DP_Example_on_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Label Differential Privacy Example for MNIST Image Classification

This colab demonstrate simple baseline algorithms for implementing Label Differential Privacy (Label DP). The colab is based on the [Tensorflow Privacy DP-SGD Example](https://github.com/tensorflow/privacy/blob/master/g3doc/tutorials/classification_privacy.ipynb). For an overview of Differential Privacy and Label Differential Privacy in Deep Learning applications, please see the paper [How to DP-fy ML: A Practical Guide to Machine Learning with Differential Privacy](https://arxiv.org/abs/2303.00654).

Label DP is appropriate for the scenarios where only the labels need to be protected. A typical example is in recommender systems, where the candidate items are known, but a user's preferences are sensitive information. Image classification might not be the best application scenario for Label DP. But we build this example on MNIST due to it's easy availability and popularity in machine learning research, and so that it would be easier to compare with the [DP-SGD training colab from Tensorflow Privacy](https://github.com/tensorflow/privacy/blob/master/g3doc/tutorials/classification_privacy.ipynb).

In [ ]:
#@title Install Tensorflow Privacy
!pip install tensorflow-privacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.2/365.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [ ]:
#@title Common utility functions
import tensorflow as tf
import numpy as np

import tensorflow_privacy
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy_lib
from tensorflow_privacy.privacy.analysis import compute_noise_from_budget_lib

tf.get_logger().setLevel('ERROR')

def load_mnist_data():
  (train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()
  train_data = np.array(train_data, dtype=np.float32).reshape(len(train_data), 28, 28, 1) / 255
  test_data = np.array(test_data, dtype=np.float32).reshape(len(test_data), 28, 28, 1) / 255
  train_labels = np.array(train_labels, dtype=np.int32)
  test_labels = np.array(test_labels, dtype=np.int32)
  return (train_data, train_labels), (test_data, test_labels)

def get_onehot_labels(train_labels, test_labels):
  train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
  test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)
  return train_labels, test_labels

def compile_model(learning_rate, dpsgd_config):
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8, strides=2, padding='same',
                             activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4, strides=2, padding='valid', activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10)])

  if dpsgd_config is None:
    optimizer = tf.keras.optimizers.Adam(learning_rate)
  else:
    optimizer = tensorflow_privacy.DPKerasAdamOptimizer(
        l2_norm_clip=dpsgd_config['l2_norm_clip'],
        noise_multiplier=dpsgd_config['noise_multiplier'],
        num_microbatches=dpsgd_config['num_microbatches'],
        learning_rate=learning_rate)

  loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)
  model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
  return model

## DP-SGD Training

In [ ]:
def run_dpsgd_train(privacy_budget, l2_norm_clip=1.5, epochs=3, batch_size=250,
                    learning_rate=0.001, delta=1e-5):
  (train_data, train_labels), (test_data, test_labels) = load_mnist_data()
  train_labels, test_labels = get_onehot_labels(train_labels, test_labels)
  noise_multiplier = compute_noise_from_budget_lib.compute_noise(
      len(train_data), batch_size, privacy_budget, epochs, delta, noise_lbd=1e-5)

  dpsgd_config = {'l2_norm_clip': l2_norm_clip, 'noise_multiplier': noise_multiplier,
                  'num_microbatches': batch_size}
  model = compile_model(learning_rate, dpsgd_config)
  return model.fit(train_data, train_labels, epochs=epochs,
                   validation_data=(test_data, test_labels), batch_size=batch_size)

In [ ]:
dpsgd_train_result = run_dpsgd_train(1.0)

11490434/11490434 [==============================] - 1s 0us/step
DP-SGD with sampling rate = 0.417% and noise_multiplier = 1.0188458598723718 iterated over 720 steps satisfies differential privacy with eps = 1 and delta = 1e-05.
Epoch 1/3
240/240 [==============================] - 70s 237ms/step - loss: 1.3896 - accuracy: 0.5591 - val_loss: 0.6384 - val_accuracy: 0.8015
Epoch 2/3
240/240 [==============================] - 57s 239ms/step - loss: 0.5420 - accuracy: 0.8379 - val_loss: 0.4532 - val_accuracy: 0.8698
Epoch 3/3
240/240 [==============================] - 58s 240ms/step - loss: 0.4373 - accuracy: 0.8835 - val_loss: 0.3993 - val_accuracy: 0.8961


## Label DP with Randomized Response

Randomized Response is one of the simplest method to achieve Label DP for classification problems.

**Remarks**

1. Randomized Response (RR) is actually a Local DP algorithm that privatize the training data -- it works by randomizing the label of each training example. After privitizing the training labels, the dataset can be used to train a model. The privacy cost does not depend on the number of model training epochs. One can also train multiple models on the data or release the data publicly without violating the DP guarantee.
2. Since RR works by transforming the training labels, it can be implemented without any modification to an existing non-private machine learning pipeline. However, the training dynamics with noisy labels might be quite different from non-private training, so the optimal hyperparameters might need to be changed.
3. For $K$-class classification, the probability that a training label remains unchanged after RR transformation is $e^\varepsilon/(e^\varepsilon + K-1)$, where $\varepsilon$ is the privacy budget. Therefore, RR usually does not work very well for problems with large number of classes and / or small privacy budget.


In [ ]:
def randomized_response(train_labels, privacy_budget, seed=1234, num_classes=10):
  rs = np.random.default_rng(seed)
  deltas = 1 + rs.integers(0, num_classes-1, size=len(train_labels))
  p_unchanged = 1 / (1 + (num_classes-1) * np.exp(-privacy_budget))
  deltas[rs.random(len(deltas)) <= p_unchanged] = 0
  print(f'RR with epsilon={privacy_budget}, {100*(1-(deltas == 0).sum()/len(deltas)):.2f}% of the labels are flipped')
  return (train_labels + deltas) % num_classes

def run_randomized_response(privacy_budget, epochs=10, batch_size=250, learning_rate=0.001, seed=1234):
  (train_data, train_labels), (test_data, test_labels) = load_mnist_data()
  train_labels = randomized_response(train_labels, privacy_budget, seed=seed)
  train_labels, test_labels = get_onehot_labels(train_labels, test_labels)
  model = compile_model(learning_rate, dpsgd_config=None)
  return model.fit(train_data, train_labels, epochs=epochs,
                   validation_data=(test_data, test_labels), batch_size=batch_size)

In [ ]:
rr_train_result = run_randomized_response(1.0)

RR with epsilon=1.0, 76.69% of the labels are flipped
Epoch 1/10
240/240 [==============================] - 4s 6ms/step - loss: 2.2777 - accuracy: 0.1785 - val_loss: 1.8253 - val_accuracy: 0.8509
Epoch 2/10
240/240 [==============================] - 1s 5ms/step - loss: 2.2550 - accuracy: 0.2160 - val_loss: 1.6862 - val_accuracy: 0.9135
Epoch 3/10
240/240 [==============================] - 2s 8ms/step - loss: 2.2457 - accuracy: 0.2227 - val_loss: 1.6059 - val_accuracy: 0.9366
Epoch 4/10
240/240 [==============================] - 1s 6ms/step - loss: 2.2399 - accuracy: 0.2243 - val_loss: 1.6168 - val_accuracy: 0.9323
Epoch 5/10
240/240 [==============================] - 1s 5ms/step - loss: 2.2354 - accuracy: 0.2252 - val_loss: 1.5921 - val_accuracy: 0.9350
Epoch 6/10
240/240 [==============================] - 1s 5ms/step - loss: 2.2314 - accuracy: 0.2268 - val_loss: 1.6203 - val_accuracy: 0.9292
Epoch 7/10
240/240 [==============================] - 1s 5ms/step - loss: 2.2278 - accuracy: 0

## Randomized Response with Prior

Randomized Response with Prior (RR-with-Prior) was introduced in [Deep Learning with Label Differential Privacy](https://arxiv.org/abs/2102.06062). It reduces the RR noises by utilizing a (public) prior on the label distribution. There are various ways to obtain such prior depending on the underlying problem. Here we present a simple KMeans clustering based approach to query the prior, and refer the readers to the [paper](https://arxiv.org/abs/2102.06062) for other approaches. A similar approach has been developed for regression labels in [https://arxiv.org/abs/2212.06074](https://arxiv.org/abs/2212.06074).

**Remarks**
1. This particular algorithm demonstrated here is no longer local DP because it queries aggregate label information in a central server to construct the priors. However, if local DP is required, this mechanism can also be implemented in the (interactive) local DP setting by adding noise locally when constructing the prior. This would lead to more noises being added, but could still be feasible in some applications (especially when the number of examples per class is large).
2. In the clustering based approach for querying priors, we used the raw pixel representations to run clustering, which works reasonably well for the MNIST dataset. For more challenging image datasets, it generally helps to use representations from a self-supervised learning (without access to the raw labels) trained model.

In [ ]:
import scipy
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

def cluster_train_data(train_data, n_clusters=100):
  train_data = train_data.reshape((train_data.shape[0], -1))
  train_data_low_dim = PCA(50).fit_transform(train_data)
  kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init="auto").fit(train_data_low_dim)
  return kmeans.labels_

def query_cluster_based_label_priors(cluster_id, train_labels, query_privacy_budget, n_classes=10):
  priors = np.zeros((len(train_labels), n_classes))
  for id in np.unique(cluster_id):
    histogram = np.bincount(train_labels[cluster_id == id], minlength=n_classes)
    noisy_histogram = histogram + scipy.stats.dlaplace.rvs(query_privacy_budget/2, size=n_classes)
    noisy_histogram = np.maximum(noisy_histogram, 0)  # clip negative values
    prior = noisy_histogram / np.maximum(1e-10, np.sum(noisy_histogram))  # normalize
    priors[cluster_id == id, :] = prior[np.newaxis, :]
  return priors

def rr_with_prior(prior, eps, y, rs):
  idx_sort = np.flipud(np.argsort(prior))
  prior_sorted = prior[idx_sort]
  tmp = np.exp(-eps)
  wks = [np.sum(prior_sorted[:(k+1)]) / (1 + (k-1)*tmp) for k in range(len(prior))]
  optim_k = np.argmax(wks) + 1

  adjusted_prior = np.zeros_like(prior) + tmp / (1 + (optim_k-1)*tmp)
  adjusted_prior[y] = 1 / (1 + (optim_k-1)*tmp)
  adjusted_prior[idx_sort[optim_k:]] = 0
  adjusted_prior /= np.sum(adjusted_prior)  # renorm in case y not in topk
  rr_label = rs.choice(len(prior), p=adjusted_prior)
  return rr_label

def run_randomized_response_with_prior(privacy_budget, budget_for_prior=0.1, epochs=10,
                                       batch_size=250, learning_rate=0.001, seed=1234):
  (train_data, train_labels), (test_data, test_labels) = load_mnist_data()
  # Note there are many different ways to get priors, here we use kmeans
  cluster_id = cluster_train_data(train_data, n_clusters=200)
  priors = query_cluster_based_label_priors(cluster_id, train_labels, budget_for_prior)
  remaining_budget = privacy_budget - budget_for_prior
  rs = np.random.default_rng(seed=seed)
  original_train_labels = train_labels
  train_labels = np.vectorize(lambda i: rr_with_prior(priors[i], 1.95, train_labels[i], rs))(np.arange(len(train_labels)))
  n_flipped = np.sum(train_labels != original_train_labels)
  print(f'RRWithPrior with epsilon={privacy_budget}, {100*n_flipped/len(train_labels):.2f}% of the labels are flipped')
  train_labels, test_labels = get_onehot_labels(train_labels, test_labels)
  model = compile_model(learning_rate, dpsgd_config=None)
  return model.fit(train_data, train_labels, epochs=epochs,
                   validation_data=(test_data, test_labels), batch_size=batch_size)

In [ ]:
rr_with_prior_train_result = run_randomized_response_with_prior(1.0)

RRWithPrior with epsilon=1.0, 12.48% of the labels are flipped
Epoch 1/10
240/240 [==============================] - 4s 7ms/step - loss: 0.8951 - accuracy: 0.7681 - val_loss: 0.2582 - val_accuracy: 0.9410
Epoch 2/10
240/240 [==============================] - 1s 5ms/step - loss: 0.5617 - accuracy: 0.8693 - val_loss: 0.2291 - val_accuracy: 0.9539
Epoch 3/10
240/240 [==============================] - 1s 5ms/step - loss: 0.5296 - accuracy: 0.8746 - val_loss: 0.2310 - val_accuracy: 0.9502
Epoch 4/10
240/240 [==============================] - 1s 5ms/step - loss: 0.5052 - accuracy: 0.8782 - val_loss: 0.2375 - val_accuracy: 0.9534
Epoch 5/10
240/240 [==============================] - 1s 5ms/step - loss: 0.4906 - accuracy: 0.8796 - val_loss: 0.2294 - val_accuracy: 0.9515
Epoch 6/10
240/240 [==============================] - 1s 5ms/step - loss: 0.4766 - accuracy: 0.8824 - val_loss: 0.2493 - val_accuracy: 0.9434
Epoch 7/10
240/240 [==============================] - 1s 5ms/step - loss: 0.4672 - ac

## Final Remarks

- This colab serves as an example of how to enable Label DP in a deep learning classification pipeline. The training setups and hyperparameters might not be optimal. For simplicity, we only demonstrated two basic Label DP algorithms, please refer to  the paper [How to DP-fy ML: A Practical Guide to Machine Learning with Differential Privacy](https://arxiv.org/abs/2303.00654) for references on other Label DP algorithms suitable for different scenarios.

- Label DP results are **not** directly comparable with DP-SGD, because Label DP only protects the labels while DP-SGD protect both the inputs and labels. Therefore, intuitively Label DP should be easier under the same privacy budget ($\varepsilon$). In practice, this might not necessarily be true, depending on the specific algorithms being used in comparison. For example, (vanilla) randomized response tends to perform poorly when the number of classes are  large or when the privacy budget ($\varepsilon$) is small.
